# Edge AI Recyclable Item Classifier

This notebook trains a lightweight image classification model using TensorFlow to recognize recyclable items.

## 1. Setup and Imports

In [ ]:
!pip install tensorflow tensorflow-datasets matplotlib pillow scikit-learn

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import os

print(f"TensorFlow version: {tf.__version__}")

## 2. Prepare Dataset

For demonstration, we'll use a public dataset. In production, you would use your own labeled recyclable items dataset.

In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 10

CLASS_NAMES = ['plastic', 'paper', 'glass', 'metal', 'organic']

print(f"Target classes: {CLASS_NAMES}")

In [ ]:
data_dir = pathlib.Path('../data')

if not data_dir.exists():
    print("Creating sample data structure...")
    for split in ['train', 'val']:
        for class_name in CLASS_NAMES:
            os.makedirs(data_dir / split / class_name, exist_ok=True)
    print("Data directories created. Please add your images to the respective folders.")
else:
    print("Data directory found!")

## 3. Load and Visualize Data

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir / 'train',
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir / 'val',
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

class_names = train_ds.class_names
print(f"Found classes: {class_names}")

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")
plt.show()

## 4. Optimize Dataset Performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## 5. Create Lightweight CNN Model

In [ ]:
num_classes = len(class_names)

model = keras.Sequential([
    layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.summary()

## 6. Compile and Train Model

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

## 7. Visualize Training Results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## 8. Save Model

In [ ]:
model_save_path = '../models/recyclable_classifier.h5'
os.makedirs('../models', exist_ok=True)
model.save(model_save_path)

with open('../models/class_names.txt', 'w') as f:
    for class_name in class_names:
        f.write(f"{class_name}\n")

print(f"Model saved to {model_save_path}")

## 9. Convert to TensorFlow Lite

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

tflite_model = converter.convert()

tflite_model_path = '../models/recyclable_classifier.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

print(f"TFLite model saved to {tflite_model_path}")
print(f"Model size: {len(tflite_model) / 1024:.2f} KB")

## 10. Test TFLite Model

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:")
print(input_details)
print("\nOutput details:")
print(output_details)

In [ ]:
for images, labels in val_ds.take(1):
    test_image = images[0:1]
    test_label = labels[0:1]
    
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    predicted_class = np.argmax(output_data[0])
    confidence = output_data[0][predicted_class]
    
    print(f"True label: {class_names[test_label[0]]}")
    print(f"Predicted: {class_names[predicted_class]} ({confidence*100:.2f}% confidence)")
    
    plt.imshow(test_image[0].numpy().astype("uint8"))
    plt.title(f"Predicted: {class_names[predicted_class]}")
    plt.axis('off')
    plt.show()

## Summary

This notebook demonstrated:
1. Training a lightweight CNN for recyclable item classification
2. Converting the model to TensorFlow Lite format
3. Applying quantization for edge deployment
4. Testing the TFLite model

The resulting `.tflite` model can be deployed to:
- Raspberry Pi
- Mobile devices (Android/iOS)
- Microcontrollers
- Edge computing devices